In [1]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)

In [12]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
for results_file in list_results_files:
    print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
    for results in lst_results:
        results_json = json.loads(results[0])
        max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
        lr = results_json["hyperparams"]["LEARNING_RATE"]
        train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
        model_name = results_json["programsettings"]["MODEL_NAME"]
        all_results.append([model_name,lr, train_batch,max_seq,results[1], results[2], results[3], results[4], results[5],results[6] ])

columns = ['model_name', 'lr', 'train_batch_size','train_max_seq','train_loss', 'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score']

df_results = pd.DataFrame(all_results, columns = columns)
df_results

reports\re\multi_model_experiment_results_2020-11-22 10_44_11_756583.pkl


,model_name,lr,train_batch_size,train_max_seq,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score
0,BioBERT_fc,0.000010,12,128,0.008484,0.061662,0.995074,0.994248,0.981641,0.969833
1,BioBERT_fc,0.000008,12,128,0.008790,0.056426,0.994977,0.993931,0.981643,0.968364
2,BioBERT_fc,0.000013,12,128,0.008213,0.060076,0.995146,0.994219,0.982797,0.972130
3,BioBERT_fc,0.000013,12,256,0.008129,0.063987,0.995194,0.994399,0.981838,0.969729
4,BioBERT_fc,0.000013,24,256,0.008214,0.057145,0.995291,0.994401,0.982028,0.971121


In [ ]:
[str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds